In [2]:
import pandas as pd

In [3]:
LABEL = "./data/covid_labels.csv"

In [4]:
label = pd.read_csv(LABEL)
label

,Id,Predicted
0,0,0
1,1,0
2,2,1
3,3,0
4,4,1
...,...,...
17453,17453,1
17454,17454,0
17455,17455,1
17456,17456,0


In [5]:
covid_df = pd.read_csv("./data/covid_bert_data.csv")
covid_df

,Unnamed: 0,source,replies
0,0,"According to the New York Times, Warner Bros. ...",NaN
1,1,Hurricane Hanna has made landfall in Texas.\n\...,@TexasTribune Guess what the cause of death is...
2,2,Monkeys on the loose in India with stolen coro...,NaN
3,3,NaN,@BelAkinyii When was the last time you washed ...
4,4,"“If Trump felt comfortable having it here, the...",@HeidiNBC These Trump fans have a right to att...
...,...,...,...
17453,17453,I wonder how many lives could’ve been saved if...,@funder Wonder how many lives could have been ...
17454,17454,The @thetimes front page on 17th March. The fi...,@NadineDorries @thetimes Inadequate supplies o...
17455,17455,Trump just completed the racism trifecta in a ...,@DNCWarRoom Fact check: Chinese is not a race....
17456,17456,Here are a few of my photographs from today’s ...,@Jess__Taylor__ @davidallengreen Eck! What are...


In [6]:
covid_df["label"] = label["Predicted"]
covid_df.fillna("None", inplace=True)
covid_df

,Unnamed: 0,source,replies,label
0,0,"According to the New York Times, Warner Bros. ...",None,0
1,1,Hurricane Hanna has made landfall in Texas.\n\...,@TexasTribune Guess what the cause of death is...,0
2,2,Monkeys on the loose in India with stolen coro...,None,1
3,3,None,@BelAkinyii When was the last time you washed ...,0
4,4,"“If Trump felt comfortable having it here, the...",@HeidiNBC These Trump fans have a right to att...,1
...,...,...,...,...
17453,17453,I wonder how many lives could’ve been saved if...,@funder Wonder how many lives could have been ...,1
17454,17454,The @thetimes front page on 17th March. The fi...,@NadineDorries @thetimes Inadequate supplies o...,0
17455,17455,Trump just completed the racism trifecta in a ...,@DNCWarRoom Fact check: Chinese is not a race....,1
17456,17456,Here are a few of my photographs from today’s ...,@Jess__Taylor__ @davidallengreen Eck! What are...,0


# Sentiment Analysis

In [1]:
from transformers import pipeline
import torch
sentiment_analysis = pipeline(model="cardiffnlp/twitter-roberta-base-sentiment-latest")

/home/lee/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Downloading: 100%|█████████████████████| 929/929 [00:00<00:00, 5.56MB/s]
Downloading: 100%|███████████████████| 478M/478M [01:19<00:00, 6.29MB/s]
Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exa

In [7]:
covid_df["Source_Sentiment"] = covid_df["source"].astype(str).apply(sentiment_analysis)

In [8]:
covid_df["Source_Sentiment"]

0        [{'label': 'Neutral', 'score': 0.5901542305946...
1        [{'label': 'Negative', 'score': 0.711997687816...
2        [{'label': 'Negative', 'score': 0.560815215110...
3        [{'label': 'Neutral', 'score': 0.6315737962722...
4        [{'label': 'Neutral', 'score': 0.7095636129379...
                               ...                        
17453    [{'label': 'Negative', 'score': 0.803073763847...
17454    [{'label': 'Neutral', 'score': 0.7463080883026...
17455    [{'label': 'Negative', 'score': 0.729388594627...
17456    [{'label': 'Neutral', 'score': 0.8799181580543...
17457    [{'label': 'Negative', 'score': 0.515761017799...
Name: Source_Sentiment, Length: 17458, dtype: object

In [48]:
covid_df["Replies_Sentiment"] = covid_df["replies2"].astype(str).apply(sentiment_analysis)

In [51]:
covid_df.to_csv("covid_sentiment.csv")

In [35]:
def trunc_text(string_of_words, size):
    ## Truncate the text to size of word! blablabla
    list_words = string_of_words.split(" ")
    if len(list_words) > size: list_words = list_words[:size]
    return " ".join(list_words)

In [36]:
trunc_text("fsda dasf dsafa sdfa sdf", 4)

'fsda dasf dsafa sdfa'

In [47]:
covid_df["replies2"] = covid_df["replies"].apply(trunc_text, size=50)

## Get Separate Replies

In [49]:
def convert_replies_id_to_sorted_text(list_of_id, data_df):
    """Convert a list of tweet ids to a list of its corresponding text in data_df
    Sort by tweets original create time"""
    if type(list_of_id) is not list:
        # if input is a single id convert type
        list_of_id = [list_of_id]
    # Select dataframe based on a list of ids
    selected_df = data_df[data_df['id'].astype(str).isin(list_of_id)]
    # Sort by their created time
    sorted_df = selected_df.sort_values(by = ["created_at"])
    # Select text fields only
    text_list = sorted_df["text"].to_list()
    return text_list


def remove_URL(original):
    """Remove url link in the text"""
    result = re.sub(r"http\S+", "", original)
    result = re.sub(r"www.\S+", "", result)
    result = re.sub(r"wasap.my+", "", result)
    return result

def join_data_id_label_v2(LABEL_PATH, ID_PATH, DATA_PATH):
    # Process Labels
    if LABEL_PATH:
        # 1: Rumour
        # 0: NonRumour
        with open(LABEL_PATH, "r") as f:
            y_label = f.read().strip().split("\n") # remove next line
        y_label = pd.DataFrame(y_label, columns = ["label"])
        y_label[y_label["label"]=="rumour"] = 1
        y_label[y_label["label"]=="nonrumour"] = 0

    ## Get Dataframe Id, with first id as source Id, and values as replies, not using dict since we have duplicated keys
    total_id_list = []
    with open(ID_PATH, "r") as f:
        for line in f:
            line = line.strip().split(',') # remove next line
            source_id = line[0]
            if len(line) > 1:
                # if we have replies id
                replies_id = line[1:]
            else:
                replies_id = []
            row = [source_id, replies_id]
            total_id_list.append(row)
    len(total_id_list)
    
    ## Create a dataframe containing a list of replies
    source_df = pd.DataFrame(total_id_list, columns = ['source_id', 'replies_id'])
    data_df = pd.read_json(DATA_PATH)
    source_df["reply_text_list"] = source_df["replies_id"].apply(convert_replies_id_to_sorted_text, data_df = data_df)
    source_df["source_text"] = source_df["source_id"].apply(convert_replies_id_to_sorted_text, data_df = data_df)

    if LABEL_PATH:
      source_df["label"] = y_label
    return source_df

def bert_preprocess(IDS, DATA, LABELS=False):
    """Function to combine all the preprocessing steps"""
    data = join_data_id_label_v2(LABELS, IDS, DATA)
    ## 1. Only keep english tweets as most of them are in english
    # Use only text data and remove URLs
    #data["source"] = data["source_text"].apply(" ".join).apply(p.clean) 
    #data["replies"] = data["reply_text_list"].apply(" ".join).apply(p.clean)
    data["source"] = data["source_text"].apply(" ".join)
    data["replies"] = data["reply_text_list"].apply(" ".join)
    
    
    if LABELS:
      data = data[['source','replies','label']]
    else:
      data = data[['source','replies']]
    
    return data

In [ ]:
DATA_PATH = "./data/full_covid.json"
ID_PATH = "./data/covid.data.txt"
full_covid = bert_preprocess(ID_PATH, DATA_PATH)
full_covid